## Using FinBERT Model trained on our data to produce Quinduple Sentiment Categories

Following codes were executed to produce the Quinduple Sentiment Categories which are to be merged with the financials - pricing and volume data

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import datetime
from datetime import timedelta, timezone
import sys
import pickle as pickle

from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from torch.utils.data import DataLoader
from transformers import AdamW
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset


## Deeplearning/ML
from sklearn.model_selection import train_test_split
from sklearn import svm, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, fbeta_score, jaccard_score, log_loss, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import tensorflow.keras.layers as L
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from yellowbrick.classifier import ROCAUC, ConfusionMatrix, PrecisionRecallCurve


In [ ]:
from google.colab import drive
drive.mount("/content/drive")
drive_path = '/content/drive/My Drive/Stock_Data/'

df = pd.read_csv(drive_path + 'Combined_FAANG_percentage_2.2.csv')

Mounted at /content/drive


In [ ]:
#df = df.sample(n=100, random_state=1)  # random_state for reproducibility
#len(sampled_df['message'])

In [ ]:
### LOAD MODEL FROM HERE
###Step 1
from torch.utils.data import SequentialSampler
# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("/content/drive/My Drive/Model/")
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
### Step 2
### takes 13min
# Encode the labels
le = LabelEncoder()
df['encoded_labels'] = le.fit_transform(df['label'])

# Ensure all data in 'Body' is a string and not empty
df['message'] = df['message'].astype(str)  # Convert to string in case there are non-string types
df = df[df['message'].str.strip().ne('')]  # Remove any rows with empty strings

# Tokenize the text data
encoded_data_test = tokenizer.batch_encode_plus(
    df['message'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

input_ids_test = torch.tensor(encoded_data_test['input_ids'])
attention_masks_test = torch.tensor(encoded_data_test['attention_mask'])
labels_test = torch.tensor(df['encoded_labels'].values)



In [ ]:
### Step 3
# Create a DataLoader
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)
dataloader_test = DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=32)

# Evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from torch.utils.data import DataLoader
import torch
import numpy as np
from tqdm import tqdm

# Assuming 'tokenizer' and 'model' are already defined and properly set up.
# Also assuming 'sampled_df' is your DataFrame.

# Convert messages to a list
sentences = list(df['message'])

sentiment_scores = []
trainloader = DataLoader(sentences, batch_size=128, shuffle=False)

# Perform sentiment analysis
with torch.no_grad():
    for batch in tqdm(trainloader):
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to('cuda')
        outputs = model(**inputs)
        logits = outputs.logits
        sentiments = torch.nn.functional.softmax(logits, dim=-1)
        sentiment_scores.extend(sentiments.cpu().numpy())

# Convert sentiment scores to a numpy array
sentiment_array = np.array(sentiment_scores)

# Create new DataFrame from sentiment_array
sentiment_df = pd.DataFrame(sentiment_array, columns=['Bearish', 'Nay', 'Neutral', 'Bullish', 'To the Moon!!'])

# Concatenate the new DataFrame with the original DataFrame
df = pd.concat([df.reset_index(drop=True), sentiment_df], axis=1)

import pickle

# Path to your Google Drive
drive_path = '/content/drive/My Drive/011223_Output_'

# Serialize the data structures to a file in your Google Drive
with open(drive_path + 'Combined_FAANG_percentage_2.2_w_sentiment_labels', 'wb') as file:
    pickle.dump((df), file)



100%|██████████| 20054/20054 [46:36<00:00,  7.17it/s]


Dataframe obtained from the above process

In [ ]:
df

,symbol,message,datetime,user,message_id,Date,Time,label,encoded_labels,Bearish,Nay,Neutral,Bullish,To the Moon!!
0,AAPL,peak profit last 6 expired option alerts aapl ...,2020-07-19 09:49:35,1442893,229008387,2020-07-19,09:49:35,1,2,0.000034,0.000191,0.000602,0.998740,0.000433
1,AAPL,aapl jul 17 382 50 calls option volume 144 44 ...,2020-07-19 09:47:26,1442893,229008357,2020-07-19,09:47:26,1,2,0.999616,0.000251,0.000069,0.000046,0.000019
2,AAPL,tsla market true bubble territory profitable c...,2020-07-19 09:01:25,1115913,229007569,2020-07-19,09:01:25,1,2,0.000016,0.000021,0.000015,0.000268,0.999680
3,AAPL,aapl analyzed 26 analysts buy consensus 86 ana...,2020-07-19 08:13:00,47688,229006733,2020-07-19,08:13:00,1,2,0.000727,0.271107,0.650643,0.077305,0.000218
4,AAPL,aapl new article dogs dow august 4 adopt ignore,2020-07-19 07:54:05,1555408,229006403,2020-07-19,07:54:05,1,2,0.000508,0.998982,0.000354,0.000119,0.000037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566821,NFLX,get long nflx stop 44 50,2009-09-22 12:21:48,78,591710,2009-09-22,12:21:48,0,1,0.000079,0.999566,0.000236,0.000070,0.000050
2566822,NFLX,nflx awards 1 mill better movie search engine ...,2009-09-21 17:44:20,384,589527,2009-09-21,17:44:20,0,1,0.000010,0.000029,0.000023,0.000652,0.999286
2566823,NFLX,39 mbt 47 swapped nflx,2009-09-21 15:48:19,78,588855,2009-09-21,15:48:19,0,1,0.000032,0.000056,0.999642,0.000220,0.000050
2566824,NFLX,long nflx,2009-09-21 15:39:57,78,588781,2009-09-21,15:39:57,0,1,0.000010,0.000020,0.999809,0.000126,0.000034


In [ ]:
df.to_csv('Combined_FAANG_percentage_2.2_w_sentiment_labels.csv')

In [ ]:
from google.colab import files
files.download('Combined_FAANG_percentage_2.2_w_sentiment_labels.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>